In [1]:
import numpy as np

In [2]:
np.convolve([3], [1, 2])
np.convolve([3, 2], [1, 2]) # flip 하고 곱한다

array([3, 8, 4])

In [3]:
np.correlate([3], [1, 2]) # 그냥 곱한다.

array([6, 3])

In [4]:
np.correlate([3, 2], [1, 2])

array([7])

우리의 목적은 필터(커널) 찾아내는 것이기 때문에, correlation 방식으로 사용해도 아무 상관 없다. 근데 신호처리하는 사람들은 이걸 아냐 모르느냐 따라서 아주 민감하다.

그런데 conv는 2번 연산을 해야 하는 반면, corr는 1번 연산을 해야 하기 때문에 비교적 간단하다. 연산이 한 번 줄어든다. 그래서 corr을 하지만, 부를 때는 conv라고 부른다.

# 신호처리 컨볼루션 연산

In [5]:
from scipy.signal import convolve, convolve2d

In [6]:
convolve([[1, 2], [1, 1]], [[3, 4], [3, 4]])

array([[ 3, 10,  8],
       [ 6, 17, 12],
       [ 3,  7,  4]])

In [7]:
convolve([1, 2], [3, 4, 5])

array([ 3, 10, 13, 10])

In [8]:
# 1, 1   3, 4
# 2, 1   3, 4

 - 먼저 열과 행이 다 플립된다.
 - 왼쪽 것이 하나씩 스트라이드 되면서, element wise방식으로 곱해진다고 생각하면 된다.

# Convolution filter visualization

https://keras.io/examples/conv_filter_visualization/

케라스는 불러올 때 tensorflow.keras로 불러와야 한다는 것 명심.

In [9]:
import time
import numpy as np
from PIL import Image as pil_image
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.preprocessing.image import save_img
from tensorflow.keras import layers
from tensorflow.keras.applications import vgg16
from tensorflow.keras import backend as K
import tensorflow as tf

Instructions for updating:
non-resource variables are not supported in the long term


In [10]:
tf.sqrt

<function tensorflow.python.ops.gen_math_ops.sqrt(x, name=None)>

In [11]:
K.mean

<function tensorflow.python.keras.backend.mean(x, axis=None, keepdims=False)>

In [12]:
tf.keras.backend.epsilon() # 아주 작은 숫자를 리턴한다. 1.x과 2.x 모두 동일!

1e-07

In [13]:
def normalize(x):
    """utility function to normalize a tensor.

    # Arguments
        x: An input tensor.

    # Returns
        The normalized input tensor.
    """
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

In [14]:
def deprocess_image(x):
    """utility function to convert a float array into a valid uint8 image.

    # Arguments
        x: A numpy-array representing the generated image.

    # Returns
        A processed numpy-array, which could be used in e.g. imshow.
    """
    # normalize tensor: center on 0., ensure std is 0.25
    x -= x.mean()
    x /= (x.std() + K.epsilon()) # 표준화 공식하고 똑같다.
    x *= 0.25 # 표준편차를 1- > 0.25에 가깝게 한다. 유동변동성을 작게 하기 위해서.

    # clip to [0, 1] # 자르는 것
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0)) # 채널퍼스트면 채널라스트 방식으로 바꿈
    x = np.clip(x, 0, 255).astype('uint8') # unit로 바꾸는 기법.
    return x

In [15]:
np.clip(np.array([1, 2, 3, 4, 5]), 1, 2) # 조건에 따라서 잘라준다.

array([1, 2, 2, 2, 2])

In [16]:
def process_image(x, former):
    """utility function to convert a valid uint8 image back into a float array.
       Reverses `deprocess_image`.

    # Arguments
        x: A numpy-array, which could be used in e.g. imshow.
        former: The former numpy-array.
                Need to determine the former mean and variance.

    # Returns
        A processed numpy-array representing the generated image.
    """
    if K.image_data_format() == 'channels_first':
        x = x.transpose((2, 0, 1))
    return (x / 255 - 0.5) * 4 * former.std() + former.mean()

In [17]:
def _generate_filter_image(input_img,
                               layer_output,
                               filter_index):
        """Generates image for one particular filter.

        # Arguments
            input_img: The input-image Tensor.
            layer_output: The output-image Tensor.
            filter_index: The to be processed filter number.
                          Assumed to be valid.

        #Returns
            Either None if no image could be generated.
            or a tuple of the image (array) itself and the last loss.
        """
#         s_time = time.time() # 시간측정 용도

        # we build a loss function that maximizes the activation
        # of the nth filter of the layer considered
#         if K.image_data_format() == 'channels_first':
#             loss = K.mean(layer_output[:, filter_index, :, :])
#         else:
        loss = K.mean(layer_output[:, :, :, filter_index])

        # we compute the gradient of the input picture wrt this loss
        grads = K.gradients(loss, input_img)[0] # 옛날방식으로 미분계산

        # normalization trick: we normalize the gradient
        grads = normalize(grads)

        # this function returns the loss and grads given the input picture
        iterate = K.function([input_img], [loss, grads]) # 2.0의 Gradient Tape

        # we start from a gray image with some random noise
        intermediate_dim = tuple(
            int(x / (upscaling_factor ** upscaling_steps)) for x in output_dim)
        if K.image_data_format() == 'channels_first':
            input_img_data = np.random.random(
                (1, 3, intermediate_dim[0], intermediate_dim[1]))
        else:
            input_img_data = np.random.random(
                (1, intermediate_dim[0], intermediate_dim[1], 3))
        input_img_data = (input_img_data - 0.5) * 20 + 128

        # Slowly upscaling towards the original size prevents
        # a dominating high-frequency of the to visualized structure
        # as it would occur if we directly compute the 412d-image.
        # Behaves as a better starting point for each following dimension
        # and therefore avoids poor local minima
        for up in reversed(range(upscaling_steps)):
            # we run gradient ascent for e.g. 20 steps
            for _ in range(epochs):
                loss_value, grads_value = iterate([input_img_data])
                input_img_data += grads_value * step

                # some filters get stuck to 0, we can skip them
                if loss_value <= K.epsilon():
                    return None

            # Calculate upscaled dimension
            intermediate_dim = tuple(
                int(x / (upscaling_factor ** up)) for x in output_dim)
            # Upscale
            img = deprocess_image(input_img_data[0])
            img = np.array(pil_image.fromarray(img).resize(intermediate_dim,
                                                           pil_image.BICUBIC))
            input_img_data = np.expand_dims(
                process_image(img, input_img_data[0]), 0)

        # decode the resulting input image
        img = deprocess_image(input_img_data[0])
#         e_time = time.time() # end time. 시간 측정용
        print('Costs of filter {:3}: {:5.0f}'.format(filter_index,
                                                                  loss_value))
        return img, loss_value

In [18]:
# gradient 예시
x = tf.ones((2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)

In [19]:
a = tf.constant([1, 2, 3])
b = tf.keras.backend.function([a], [np.mean])
# tf.keras.backend.function()

TypeError: Can not convert a function into a Tensor or Operation.

In [20]:
a = next(reversed([3, 2, 1]))

https://tensorspace.org/

convolution할때 아주 기똥차다. playground들어가면 아주 좋은거 많다.

In [21]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [22]:
dir(model.layers)
dir(model.get_layer) #name을 지정할 수 있다. 특정 레이어를 뽑을 수 있다.

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

### 그냥 바로 실습

In [23]:
vgg = vgg16.VGG16(weights='imagenet', include_top=False)

In [24]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [25]:
import MapFilter

In [26]:
MapFilter.visualize_layer(vgg, 'block3_conv3')
# visualize_layer(vgg, 'block3_conv3')

Compute filters 0 to 256
Costs of filter   0:  1064 ( 6.28s )
Costs of filter   1:  3045 ( 1.30s )
Costs of filter   2:  1344 ( 1.40s )
Costs of filter   3:  3989 ( 1.29s )
Costs of filter   4:   761 ( 1.37s )
Costs of filter   5:  3284 ( 1.57s )
Costs of filter   6:  1044 ( 1.31s )
Costs of filter   7:  1634 ( 1.33s )
Costs of filter   8:  1619 ( 1.41s )
Costs of filter   9:  3113 ( 1.41s )
Costs of filter  10:   969 ( 1.40s )
Costs of filter  11:  2909 ( 1.44s )
Costs of filter  12:  1568 ( 1.31s )
Costs of filter  13:  1062 ( 1.47s )
Costs of filter  14:  1211 ( 1.41s )
Costs of filter  15:   888 ( 1.54s )
Costs of filter  16:  1323 ( 1.37s )
Costs of filter  17:   722 ( 1.34s )
Costs of filter  18:  2603 ( 1.52s )
Costs of filter  19:  1779 ( 1.35s )
Costs of filter  20:  1053 ( 1.46s )
Costs of filter  21:  2801 ( 1.51s )
Costs of filter  22:  1103 ( 1.52s )
Costs of filter  23:  2937 ( 1.45s )
Costs of filter  24:   936 ( 1.35s )
Costs of filter  25:  1342 ( 1.40s )
Costs of filt

Costs of filter 229:  1213 ( 2.09s )
Costs of filter 230:  2739 ( 1.99s )
Costs of filter 231:   934 ( 2.10s )
Costs of filter 232:  1076 ( 2.05s )
Costs of filter 233:  1486 ( 2.13s )
Costs of filter 234:  1740 ( 2.11s )
Costs of filter 235:  1310 ( 1.96s )
Costs of filter 236:  2363 ( 2.09s )
Costs of filter 237:  2676 ( 2.12s )
Costs of filter 238:  1411 ( 1.96s )
Costs of filter 239:  1589 ( 1.99s )
Costs of filter 240:  1500 ( 2.07s )
Costs of filter 241:   476 ( 2.00s )
Costs of filter 242:  1282 ( 2.15s )
Costs of filter 243:   359 ( 2.05s )
Costs of filter 244:  1094 ( 1.95s )
Costs of filter 245:  1276 ( 2.05s )
Costs of filter 246:  1349 ( 2.04s )
Costs of filter 247:  1272 ( 1.96s )
Costs of filter 248:   248 ( 2.13s )
Costs of filter 249:  1572 ( 2.17s )
Costs of filter 250:  1050 ( 2.04s )
Costs of filter 251:  1333 ( 1.95s )
Costs of filter 252:  1229 ( 1.96s )
Costs of filter 253:   793 ( 2.02s )
Costs of filter 254:  2551 ( 2.00s )
247 filter processed.


### transfer 4가지 전략

내일!

### 남이 만든 모델 쓰는 법

1. 텐서플로상에서 가져오는 방법 (2.x): 텐서허브에서 가져온다. 텐서허브는 사람들이 모델 모아두는 곳이다.
2. 케라스에서 가져오는 방법

In [27]:
tf.keras.applications # 남이 만들어둔 모델 가져다 쓸 수 있다.

<module 'tensorflow_core.keras.applications' from 'C:\\Users\\white\\Anaconda3\\envs\\tf2.0-gpu\\lib\\site-packages\\tensorflow_core\\python\\keras\\api\\_v2\\keras\\applications\\__init__.py'>

In [ ]:
vgg = tf.keras.applications.vgg16.VGG16() # weight, include_top